# More data viz ideas 
- now that I have a basic layout, we can have more concrete ideas

In [4]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import sheworewhat as sww

In [164]:
df = sww.closet_df()
worn_df = sww.complete_df(df)
worn_df.head(5)

,ID,Name,Count,Item,Category,Sub-Category,Color,Pattern,Brand,Bought,Cost,2023,Price
0,0,Zara Turtleneck,3,Turtleneck,Top,Sweater,Black,Plain,Zara,"Secondhand, Thrifted",cheap,No,0.0
1,1,Plisse Tropical Tank,0,Tropical Tank,Top,Tanktop,"Black, Red, Gold",Feather,Plisse,"Secondhand, Thrifted",cheap,No,0.0
2,2,Plisse Leopard Tank,2,Leopard Tank,Top,Tanktop,"Black, Tan",Leopard,Plisse,"Secondhand, Thrifted",cheap,No,15.0
3,3,Aerie Jeans,10,Jeans,Bottom,Pants,Blue,Plain,Aerie,New,cheap,No,0.0
4,4,Free People Shirt,1,Shirt,Top,Shirt,"Black, White",Cheetah,Free People,"Secondhand, Depop",cheap,No,0.0


Ideas:
- Seasonal shit 
- Color analysis

## Color Analysis
- would be SICK to actually plot the colors of my closet

In [96]:
# split the color 
color_df = pd.concat([worn_df[["ID", "Item", "Count"]], worn_df["Color"].str.split(",", expand=True)], axis=1)
color_df = color_df.rename(columns={0:"Color1",
                                   1:"Color2", 
                                   2:"Color3"})
color_df.sample(1)

In [85]:
plot_primaryc = alt.Chart(color_df, 
                          title="Primary Color"
                ).mark_bar(
                    cornerRadiusBottomRight=10,
                    cornerRadiusTopRight=10
                ).encode(
                    alt.Y(
                        "Color1",
                        sort="-x",
                        title="Primary Color of Wardrobe"
                    ),
                    alt.X(
                        "count()",
                        title="Count"
                    ),
                    color="Color1",
                    tooltip=["count()"]
                )
plot_primaryc

What if I used a stacked bar chart?

In [80]:
plot_secondc = alt.Chart(color_df, 
                          title="Secondary Color"
                ).mark_bar(
                    cornerRadiusBottomRight=10,
                    cornerRadiusTopRight=10
                ).encode(
                    alt.Y(
                        "Color2",
                        sort="-x",
                        title="Primary Color of Wardrobe"
                    ),
                    alt.X(
                        "count()",
                        title="Count"
                    ),
                    color="Color1",
                    tooltip=["count()"]
                )
plot_secondc

was going to try to melt all the colors into one column but i cannot get it to work

In [106]:
worn_df.columns

In [46]:
# split the color column and melt into one
color_df = pd.concat([worn_df[["ID", "Item", "Count", "Category"]], 
                      worn_df["Color"].str.split(",", n=2, expand=True)], 
                     axis=1)

color_df = color_df.melt(id_vars=["ID"], 
                         value_vars=[0, 1, 2],
                         var_name="former_name",
                         value_name="Color"
                        ).fillna("Delete")

# only keep multiple colors if present
color_df = color_df[color_df["Color"] != "Delete"]

plot_color = alt.Chart(color_df, 
                          title="Primary Color"
                ).mark_bar(
                    cornerRadiusBottomRight=10,
                    cornerRadiusTopRight=10
                ).encode(
                    alt.Y(
                        "Color",
                        sort="-x",
                        title="Primary Color of Wardrobe"
                    ),
                    alt.X(
                        "count()",
                        title="Count"
                    ),
                    tooltip=["count()"]
                )
plot_color

alt.Chart(...)

In [83]:
base = alt.Chart(color_df).encode(
    theta=alt.Theta("count()", stack=True),
    color=alt.Color("Color",
                   scale=alt.Scale(
                    range=["#1a1919", # black
                           "#6b6054", # brown 
                           "#dbbb07", # gold 
                           "#056e0e", # green
                           "#3c3f61", # navy
                           "#e872cc", # pink,
                           "#ff2930", # red
                           "#bdb3b4", # silver
                           "#946e46", # tan
                           "#ffffff", # white
                           "#edc59a", # beige
                           
                          ] 
                    )
                   ),
    tooltip=["Color", "count()"]
)

c1 = base.mark_arc(innerRadius=10, stroke="#e0ddd5")
c1 

alt.Chart(...)

The colors ae being double counted!!! why are there two blacks

In [87]:
color_df = pd.concat([worn_df[["ID", "Item", "Count", "Category"]], 
                      worn_df["Color"].str.split(",", n=2, expand=True)], 
                     axis=1)

color_df = color_df.melt(id_vars=["ID"], 
                         value_vars=[0],
                         var_name="former_column",
                         value_name="Color"
                        ).fillna("Delete"
                        )

# only keep multiple colors if present
color_df = color_df[color_df["Color"] != "Delete"] #.drop(["ID", "former_name"], axis=1)
color_df

# x = pd.DataFrame(color_df["Color"])
# x

,ID,former_column,Color
0,0,0,Black
1,1,0,Black
2,2,0,Black
3,3,0,Blue
4,4,0,Black
...,...,...,...
87,87,0,Black
88,88,0,Black
89,89,0,Silver
90,90,0,Black


In [4]:
color_df = pd.concat([worn_df[["ID", "Item", "Count", "Category"]], 
                      worn_df["Color"].str.split(",", n=2, expand=True)], 
                     axis=1)

color_df = color_df.melt(id_vars=["ID"], 
                         value_vars=[0],
                         var_name="former_column",
                         value_name="Color"
                        ).fillna("Delete"
                        )

# only keep multiple colors if present
color_df = color_df[color_df["Color"] != "Delete"]

base = alt.Chart(color_df,
                title = "Color Composition of 2023 Closet").encode(
    theta=alt.Theta("count()", stack=True),
    color=alt.Color("Color",
                   scale=alt.Scale(
                    range=["#edc59a", # beige
                           "#1a1919", # black
                           "#455ad1", # blue
                           "#422d08", # brown 
                           "#6e353a", # burgundy
                           "#9c501e", # clay
                           "#ffffcc", # cream
                           "#dbbb07", # gold 
                           "#6e6e6e", # gray
                           "#056e0e", # green
                           "#c2addb", # lavender
                           "#243763", # navy
                           "#fa9c3e", # orange
                           "#e872cc", # pink,
                           "#ff2930", # red
                           "#bdb3b4", # silver
                           "#ab8f72", # tan
                           "#ffffff", # white
                           "#faf20a", # yellow
                          ]
                       ), legend=None
                   ),
    tooltip=["Color", "count()"]
)

plot_color = base.mark_arc(innerRadius=1) #, stroke="#e3e1de")
plot_color

alt.Chart(...)

In [6]:
color_df

,ID,former_column,Color
0,0,0,Black
1,1,0,Black
2,2,0,Black
3,3,0,Blue
4,4,0,Black
...,...,...,...
87,87,0,Black
88,88,0,Black
89,89,0,Silver
90,90,0,Black


In [41]:
def plot_color(worn_df):
    """
    Function for color composition of color.

    Parameters:
    -----------
        worn_df : pandas.DataFrame

    Returns:
    --------
        plot : altair.Chart
            Pie chart of colors present in closet.
    """
    color_df = pd.concat(
        [
            worn_df[["ID", "Item", "Count", "Category"]],
            worn_df["Color"].str.split(",", n=2, expand=True),
        ],
        axis=1,
    )

    color_df = color_df.melt(
        id_vars=["ID"], value_vars=[0], var_name="former_column", value_name="Color"
    ).fillna("Delete")

    # only keep multiple colors if present
    color_df = color_df[color_df["Color"] != "Delete"]

    base = alt.Chart(color_df, title="Color Composition of 2023 Closet").encode(
        theta=alt.Theta("count()", stack=True),
        color=alt.Color(
            "Color",
            scale=alt.Scale(
                range=[
                    "#edc59a",  # beige
                    "#1a1919",  # black
                    "#455ad1",  # blue
                    "#422d08",  # brown
                    "#6e353a",  # burgundy
                    "#9c501e",  # clay
                    "#ffffcc",  # cream
                    "#dbbb07",  # gold
                    "#6e6e6e",  # gray
                    "#056e0e",  # green
                    "#c2addb",  # lavender
                    "#243763",  # navy
                    "#fa9c3e",  # orange
                    "#e872cc",  # pink,
                    "#ff2930",  # red
                    "#bdb3b4",  # silver
                    "#ab8f72",  # tan
                    "#ffffff",  # white
                    "#faf20a",  # yellow
                ]
            ),
            legend=None,
        ),
        tooltip=["Color", "count()"],
    )

    plot_color = base.mark_arc(innerRadius=0, opacity=0.80).configure_view(strokeWidth=0)  # , stroke="#e3e1de")
    return plot_color

In [104]:
plot_color(worn_df)

alt.Chart(...)

## General closet composition

In [34]:
df = sww.closet_df()
worn_df = sww.complete_df(df)
worn_df.head(5)

,ID,Name,Count,Item,Category,Sub-Category,Color,Pattern,Brand,Bought,Cost,2023,Price
0,0,Zara Turtleneck,3,Turtleneck,Top,Sweater,Black,Plain,Zara,"Secondhand, Thrifted",cheap,No,0.0
1,1,Plisse Tropical Tank,0,Tropical Tank,Top,Tanktop,"Black, Red, Gold",Feather,Plisse,"Secondhand, Thrifted",cheap,No,0.0
2,2,Plisse Leopard Tank,2,Leopard Tank,Top,Tanktop,"Black, Tan",Leopard,Plisse,"Secondhand, Thrifted",cheap,No,15.0
3,3,Aerie Jeans,10,Jeans,Bottom,Pants,Blue,Plain,Aerie,New,cheap,No,0.0
4,4,Free People Shirt,1,Shirt,Top,Shirt,"Black, White",Cheetah,Free People,"Secondhand, Depop",cheap,No,0.0


In [38]:
base = alt.Chart(worn_df,
                 title = "Closet Categories"
                 
                ).encode(
    theta=alt.Theta("count()", stack=True),
    color="Category",
    tooltip=["Category", "count()"]
)

cat = base.mark_arc(innerRadius=0, opacity=0.85)
txt = base.mark_text(
            radius=175, 
            size=15
        ).encode(
            alt.Color(
                "Category",
                scale=alt.Scale(
                    range=["#bb8c9d","#9a8ca6","#8ba88a","#5bccc1","#e0ddd5", "#bfae8f"]
                ),
                legend=None
            ),
            text="Category")

plot_categories = cat + txt
plot_categories = plot_categories.configure_view(strokeWidth=0).configure_title(color="#706f6c") #.properties(height=200, width=400).
plot_categories

alt.LayerChart(...)

In [110]:
def plot_categories(worn_df):
    """
    Function for categorial composition of closet

    Parameters:
    -----------
        worn_df : pandas.DataFrame

    Returns:
    --------
        plot : altair.Chart
            Pie chart of clothing categories present in closet.
    """
    base = alt.Chart(worn_df,
                 title = "Closet Categoies"
                ).encode(
    theta=alt.Theta("count()", stack=True),
    color="Category",
    tooltip=["Category", "count()"])

    cat = base.mark_arc(innerRadius=0, opacity=0.85)
    txt = base.mark_text(
                radius=130, 
                size=15
            ).encode(
                alt.Color(
                    "Category",
                    scale=alt.Scale(
                        range=["#bb8c9d","#9a8ca6","#8ba88a","#5bccc1","#e0ddd5", "#bfae8f"]
                    ),
                    legend=None
                ),
                text="Category")

    plot_categories = cat + txt
    plot_categories = plot_categories.configure_view(strokeWidth=0).properties(height=200, width=600)
    return plot_categories

In [111]:
plot_categories(worn_df)

alt.LayerChart(...)

In [163]:
worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Thrifted", "Thrifted")
worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Depop", "Vintage")
worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Gifted", "Gifted")

worn_df

,ID,Name,Count,Item,Category,Sub-Category,Color,Pattern,Brand,Bought,Cost,2023,Price
0,0,Zara Turtleneck,3,Turtleneck,Top,Sweater,Black,Plain,Zara,Thrifted,cheap,No,0.0
1,1,Plisse Tropical Tank,0,Tropical Tank,Top,Tanktop,"Black, Red, Gold",Feather,Plisse,Thrifted,cheap,No,0.0
2,2,Plisse Leopard Tank,2,Leopard Tank,Top,Tanktop,"Black, Tan",Leopard,Plisse,Thrifted,cheap,No,15.0
3,3,Aerie Jeans,10,Jeans,Bottom,Pants,Blue,Plain,Aerie,New,cheap,No,0.0
4,4,Free People Shirt,1,Shirt,Top,Shirt,"Black, White",Cheetah,Free People,Depop,cheap,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,87,Target Bow,4,Bow,Accessory,Hat,Black,Plain,Target,New,cheap,No,10.0
88,88,Amazon Snake Tights,1,Snake Tights,Accessory,Tight,Black,Snake,Amazon,New,cheap,No,21.0
89,89,Francesca'S Purse,1,Purse,Accessory,Bag,Silver,Rhinestone,Francesca'S,New,pricy,No,60.0
90,90,Levi'S Jean Jacket,2,Jean Jacket,Outerwear,Coat,Black,Plain,Levi'S,New,pricy,No,0.0


In [31]:
def plot_bought(worn_df):
    """
    Function for secondhand vs new closet composition.

    Parameters:
    -----------
        worn_df : pandas.DataFrame

    Returns:
    --------
        plot : altair.Chart
            Pie chart of secondhand vs new closet composition.
    """
    worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Thrifted", "Thrifted")
    worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Depop", "Vintage")
    worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Gifted", "Gifted")

    base = alt.Chart(worn_df,
                 title = "Closet Categories"
                ).encode(
    theta=alt.Theta("count()", stack=True),
    color="Bought",
    tooltip=["Bought", "count()"]
    )

    cat = base.mark_arc(innerRadius=0, opacity=0.85)
    txt = base.mark_text(
                angle=20,
                radius=127, 
                size=15
            ).encode(
                alt.Color(
                    "Bought",
                    scale=alt.Scale(
                        range=["#bb8c9d","#9a8ca6","#8ba88a","#5bccc1","#e0ddd5", "#bfae8f"]
                    ),
                    legend=None
                ),
                text="Bought"
    )

    plot_bought = cat + txt
    plot_bought = plot_bought.configure_view(strokeWidth=0).properties(height=200, width=600).configure_title(color="#706f6c")
    return plot_bought

In [32]:
plot_bought(worn_df)

alt.LayerChart(...)

In [14]:
df = sww.closet_df()
worn_df = sww.complete_df(df)

In [45]:
def plot_bought(worn_df):
    """
    Function for secondhand vs new closet items.

    Parameters:
    -----------
        worn_df : pandas.DataFrame

    Returns:
    --------
        plot : altair.Chart
            Bar chart of secondhand vs new closet composition.
    """
    worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Thrifted", "Thrifted")
    worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Depop", "Vintage")
    worn_df["Bought"] = worn_df["Bought"].str.replace("Secondhand, Gifted", "Gifted")

    plot = alt.Chart(worn_df, title="New vs Secondhand Items").mark_bar(
        color="#bfae8f",
        cornerRadiusBottomRight=10,
        cornerRadiusTopRight=10,
        opacity=0.85
    ).encode(
        alt.X("count()", title="Count"),
        alt.Y("Bought", sort="-x"),
    ).configure_title(color="#706f6c").configure_axis(
        grid=False, domain=False, labelColor='#706f6c', titleColor="#706f6c").properties(height=200)
    return plot

In [46]:
plot_bought(worn_df)

alt.Chart(...)